In [3]:
import os

# Your dataset path (change if needed)
dataset_path = "/teamspace/studios/this_studio/asl_alphabet_train/asl_alphabet_train"

total_images = 0
folder_counts = {}

# Loop through each alphabet folder
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        image_count = len([
            f for f in os.listdir(folder_path)
            if os.path.isfile(os.path.join(folder_path, f))
        ])
        folder_counts[folder] = image_count
        total_images += image_count

# Print per folder count (optional)
for folder, count in folder_counts.items():
    print(f"{folder}: {count} images")

print(f"\n✅ Total images across all folders: {total_images}")


A: 3000 images
B: 3000 images
C: 3000 images
D: 3000 images
del: 1694 images

✅ Total images across all folders: 13694


In [1]:
! cd /teamspace/uploads


In [2]:
! pip install tensorflow matplotlib numpy tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 MB 81.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 135.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 145.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 208.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 220.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 10.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: tensorboard━━━╺━━━━━━━━━ 12/16 [astunparse]io-gcs-filesystem]
    Found existing installation: tensorboard 2.15.10m━━━━━━━━━ 12/16 [astunparse]
    Uninstalling tensorboard-2.15.1:0m╺━━━━━━━━━ 12/16 [astunparse]
      Successfully uninstalled tensorboard-2.15.1━━━━━━━━━ 12/16 [astunparse]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16/16 [tensorflow]6 [tensorflow]]

[notice] A new release of pip is available

In [3]:
# 📦 1. Imports & Setup
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

print("✅ TensorFlow version:", tf.__version__)


2025-08-02 03:50:42.204806: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-02 03:50:42.355820: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754106642.427504    5648 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754106642.446852    5648 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754106642.571210    5648 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

✅ TensorFlow version: 2.19.0


In [4]:
# 📂 2. Paths
base_path = "/teamspace/studios/this_studio/ASL_data/split_data"
train_dir = os.path.join(base_path, "train")
val_dir = os.path.join(base_path, "val")
test_dir = "/teamspace/studios/this_studio/ASL_data/unzipped/asl_alphabet_test"  # change if needed


In [5]:
# 🌀 3. ImageDataGenerators
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)


Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.
Found 28 images belonging to 1 classes.


In [6]:
# 🧠 4. Transfer Learning Model (MobileNetV2)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


I0000 00:00:1754106911.380407    5648 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20763 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:04.0, compute capability: 8.9


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,425,693 (9.25 MB)

 Trainable params: 167,709 (655.11 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [7]:
# ⛑️ 5. Callbacks: Save checkpoint + stop early if overfit
checkpoint_path = "asl_best_model.h5"

callbacks = [
    ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1),
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
]


In [8]:
# 🎯 6. Train the Model
EPOCHS = 10  # You can increase later

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=callbacks
)


Epoch 1/10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1754107011.178416    8623 service.cc:152] XLA service 0x75efb0002e50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754107011.178959    8623 service.cc:160]   StreamExecutor device (0): NVIDIA L4, Compute Capability 8.9
2025-08-02 03:56:51.394045: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1754107012.589019    8623 cuda_dnn.cc:529] Loaded cuDNN version 90701
2025-08-02 03:56:54.046766: I external/local_xla

   4/2175 ━━━━━━━━━━━━━━━━━━━━ 1:41 47ms/step - accuracy: 0.0618 - loss: 3.5658  

I0000 00:00:1754107025.579204    8623 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.8671 - loss: 0.5135

2025-08-02 03:59:24.034258: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1195', 220 bytes spill stores, 220 bytes spill loads

2025-08-02 03:59:24.275100: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1195_0', 480 bytes spill stores, 1372 bytes spill loads

2025-08-02 03:59:25.834110: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1195', 5356 bytes spill stores, 5412 bytes spill loads

2025-08-02 03:59:26.195387: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1195', 5324 bytes spill stores, 5308 bytes spill loads

2025-08-02 03:59:26.29890


Epoch 1: val_accuracy improved from None to 0.97943, saving model to asl_best_model.h5


2175/2175 ━━━━━━━━━━━━━━━━━━━━ 166s 67ms/step - accuracy: 0.9472 - loss: 0.2004 - val_accuracy: 0.9794 - val_loss: 0.0637
Epoch 2/10
2174/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.9879 - loss: 0.0385
Epoch 2: val_accuracy did not improve from 0.97943
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 123s 56ms/step - accuracy: 0.9869 - loss: 0.0404 - val_accuracy: 0.9756 - val_loss: 0.0685
Epoch 3/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.9908 - loss: 0.0278
Epoch 3: val_accuracy improved from 0.97943 to 0.98879, saving model to asl_best_model.h5


2175/2175 ━━━━━━━━━━━━━━━━━━━━ 181s 83ms/step - accuracy: 0.9907 - loss: 0.0283 - val_accuracy: 0.9888 - val_loss: 0.0349
Epoch 4/10
2174/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.9933 - loss: 0.0189
Epoch 4: val_accuracy did not improve from 0.98879
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 153s 70ms/step - accuracy: 0.9928 - loss: 0.0214 - val_accuracy: 0.9888 - val_loss: 0.0349
Epoch 5/10
2174/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9941 - loss: 0.0181
Epoch 5: val_accuracy did not improve from 0.98879
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 134s 61ms/step - accuracy: 0.9935 - loss: 0.0201 - val_accuracy: 0.9887 - val_loss: 0.0354
Epoch 6/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9937 - loss: 0.0194
Epoch 6: val_accuracy did not improve from 0.98879
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 132s 60ms/step - accuracy: 0.9940 - loss: 0.0181 - val_accuracy: 0.9876 - val_loss: 0.0387
Epoch 7/10
2174/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.9951 - loss: 0.0152
Epo

2175/2175 ━━━━━━━━━━━━━━━━━━━━ 153s 70ms/step - accuracy: 0.9949 - loss: 0.0164 - val_accuracy: 0.9940 - val_loss: 0.0180
Epoch 8/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.9958 - loss: 0.0137
Epoch 8: val_accuracy did not improve from 0.99402
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 152s 70ms/step - accuracy: 0.9956 - loss: 0.0137 - val_accuracy: 0.9924 - val_loss: 0.0243
Epoch 9/10
2174/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9948 - loss: 0.0162
Epoch 9: val_accuracy did not improve from 0.99402
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 124s 57ms/step - accuracy: 0.9956 - loss: 0.0144 - val_accuracy: 0.9937 - val_loss: 0.0206
Epoch 10/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.9955 - loss: 0.0136
Epoch 10: val_accuracy did not improve from 0.99402
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 128s 59ms/step - accuracy: 0.9956 - loss: 0.0137 - val_accuracy: 0.9936 - val_loss: 0.0210
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 7.


In [15]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load model
model = load_model("asl_best_model.h5")

# Labels from training folder (make sure they're same order as training)
class_labels = sorted(os.listdir("/teamspace/studios/this_studio/ASL_data/unzipped/asl_alphabet_train/asl_alphabet_train"))

# Test folder path (28 images, one per class)
test_dir = "/teamspace/studios/this_studio/ASL_data/unzipped/asl_alphabet_test/asl_alphabet_test"

# Init counters
correct = 0
total = 0

# Loop through all test images
for filename in os.listdir(test_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(test_dir, filename)

        # True label from filename (e.g., A_test.jpg → A)
        true_label = filename[0].upper()

        # Preprocess
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Predict
        preds = model.predict(img_array)
        pred_class = np.argmax(preds[0])
        pred_label = class_labels[pred_class]

        # Compare
        is_correct = pred_label == true_label
        if is_correct:
            correct += 1
        else:
            print(f"❌ Wrong: {filename} → Predicted: {pred_label}, Actual: {true_label}")
        
        total += 1

# Final Accuracy
accuracy = (correct / total) * 100
print(f"\n✅ Total: {total} | Correct: {correct} | Accuracy: {accuracy:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━